# **응급상황 자동 인식 및 응급실 연계 서비스**

## **1.환경설정**

#### 라이브러리 로딩

In [21]:
#!pip install openpyxl

In [22]:
#필요한 라이브러리 설치 및 불러우기
import os
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import openai
from openai import OpenAI
import json
import sys
import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification


# 조에서 생성한 모듈 불러오기 -------------
import emergency as em

In [23]:
path = os.getcwd() + "/"
sys.path.append(path)

### (1) open ai key 등록

In [ ]:
openai.api_key = em.load_file(path + '/api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key
print(os.environ['OPENAI_API_KEY'][:30])

### (2) audio to text

In [25]:
audio_path = path + 'audio/'

In [26]:
from emergency import audio2text
df = em.audio2text(audio_path)
df

,Audio,Transcribed
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...
1,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리..."
4,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...


### (3) text summary

In [27]:
df = em.text2summary(df)
df

,Audio,Transcribed,summary
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,"{""summary"": ""아빠가 계단에서 넘어져 머리에서 피가 나오고 어지럽다고 합니..."
1,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,"{""summary"": ""열이 올라서 머리가 아프고, 띵해요. 목이 좀 아픈 것 같아..."
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,"{""summary"": ""콩이 코에 들어간 응급상황 발생"", ""부위"": ""코"", ""증..."
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...","{""summary"": ""머리를 박고 3시간 지나서 어지러움과 메스꺼움이 있는 상황""..."
4,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,"{""summary"": ""미끄러져 엉덩방아를 찍어 꼬리뼈가 지속적으로 아픔"", ""부위..."


In [46]:
df_location = pd.read_excel(path + "audio_location.xlsx")
df_location.rename(columns={'filename': 'Audio'}, inplace=True)
df_location

,Audio,위도,경도,Unnamed: 3,Unnamed: 4
0,audio1.mp3,37.358618,127.115036,NaN,NaN
1,audio2.mp3,36.815571,127.128844,NaN,NaN
2,audio3.mp3,37.538435,126.989828,NaN,NaN
3,audio4.mp3,35.185745,129.076541,NaN,NaN
4,audio5.mp3,36.503544,127.252941,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,


In [29]:
df = pd.merge(df, df_location, on='Audio', how='left')
df.drop(columns=["Unnamed: 3", "Unnamed: 4"], errors="ignore", inplace=True)
df

,Audio,Transcribed,summary,위도,경도
0,audio1.mp3,지금 아빠가 넘어졌어요. 머리에서 피가 나는데 숨은 쉬고 있어요. 지금 막 일어났어...,"{""summary"": ""아빠가 계단에서 넘어져 머리에서 피가 나오고 어지럽다고 합니...",37.358618,127.115036
1,audio2.mp3,119죠. 제가 지금 열이 열이 올랐어요. 몇 도냐면은 38도 정도 돼요. 머리가 ...,"{""summary"": ""열이 올라서 머리가 아프고, 띵해요. 목이 좀 아픈 것 같아...",36.815571,127.128844
2,audio3.mp3,동생이 콩 가지고 놀다가 코에 들어가서 한쪽 코가 막혔어요. 아무리 빼보려 해도 안...,"{""summary"": ""콩이 코에 들어간 응급상황 발생"", ""부위"": ""코"", ""증...",37.538435,126.989828
3,audio4.mp3,"아까 가다가 머리를 박았는데, 처음에는 괜찮다가, 지금 3시간 정도 지났는데, 머리...","{""summary"": ""머리를 박고 3시간 지나서 어지러움과 메스꺼움이 있는 상황""...",35.185745,129.076541
4,audio5.mp3,화장실에서 미끄러워서 엉덩방아를 찍었어요. 그러고 꼬리뼈가 계속 아파요. 점점 아픈...,"{""summary"": ""미끄러져 엉덩방아를 찍어 꼬리뼈가 지속적으로 아픔"", ""부위...",36.503544,127.252941


### (4) 응급실 등급분류

In [30]:
save_directory = path + 'fine_tuned_bert_our/'
model = AutoModelForSequenceClassification.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [31]:
pred = em.model_prediction(save_directory, df)
pred

2

### (5) 응급실추천

In [32]:
emergency = pd.read_csv(path + '응급실 정보.csv')
map_key = em.load_file(path + 'map_key.txt')
map_key = json.loads(map_key)
c_id, c_key = map_key['c_id'], map_key['c_key']

In [33]:
emergency

,병원이름,주소,응급의료기관 종류,전화번호 1,전화번호 3,위도,경도
0,(의)내경의료재단울산제일병원,울산광역시 남구 남산로354번길 26 (신정동),응급실운영신고기관,052-220-3300,052-220-3334,35.548238,129.307011
1,(의)서일의료재단기장병원,부산광역시 기장군 기장읍 대청로72번길 6,지역응급의료기관,051-723-0171,051-723-2119,35.236029,129.216492
2,(의)성세의료재단 뉴성민병원,"인천광역시 서구 칠천왕로33번길 17 (석남동, 신석로 70(석남1동, 성민병원))",지역응급의료기관,032-726-1000,032-726-1190,37.508994,126.669479
3,(의)영문의료재단다보스병원,"경기도 용인시 처인구 백옥대로1082번길 18, 다보스종합병원 (김량장동)",지역응급의료센터,031-8021-2114,031-8021-2130,37.234641,127.210499
4,(의)효심의료재단용인서울병원,경기도 용인시 처인구 고림로 81 (고림동),지역응급의료기관,031-337-0114,031-336-0119,37.240316,127.214491
...,...,...,...,...,...,...,...
520,효산의료재단안양샘병원,"경기도 안양시 만안구 삼덕로 9 (안양동, 안양샘병원)",지역응급의료센터,031-467-9717,031-467-9119,37.393404,126.924477
521,효산의료재단지샘병원,"경기도 군포시 군포로 591 (당동, (G샘병원)군포샘병원)",지역응급의료센터,031-389-3000,031-389-3119,37.358645,126.947360
522,효성시티병원,부산광역시 해운대구 해운대로 135 (재송동),응급실운영신고기관,051-709-3000,051-709-3119,35.185413,129.121459
523,흑룡의원,인천광역시 옹진군 백령면 백령로 831,응급실운영신고기관,032-837-6873,032-837-3153,37.959524,124.665499


In [34]:
em.recom_em(path, df)

탐색범위:11.0km * 11.0km
(2626, '분당서울대학교병원', '031-787-2114', '031-787-3119')
(4707, '대진의료재단분당제생병원', '031-779-0114', '031-779-0119')
(5973, '국군수도병원', '031-725-6075', '031-725-6119')


In [39]:
from concurrent.futures import ThreadPoolExecutor
hospital = pd.read_csv(path + '응급실 정보.csv')
save_directory = path + "fine_tuned_bert/"

In [43]:
openai.api_key = em.load_file(path + '/api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key
df = em.audio2text(audio_path)
df = em.text2summary(df)
pred = em.model_prediction(save_directory, df)
df_location = pd.read_excel(path + "audio_location.xlsx")
df_location.rename(columns={'filename': 'Audio'}, inplace=True)
df = pd.merge(df, df_location, on='Audio', how='left')
em.recom_em(path, df)
if pred < 3 :
   df['위도'], df['경도'] = map(float, input().split())
   em.recom_em(path, df)
else : print("집에서 휴식을 취하세요")

탐색범위:11.0km * 11.0km
(2626, '분당서울대학교병원', '031-787-2114', '031-787-3119')
(4789, '대진의료재단분당제생병원', '031-779-0114', '031-779-0119')
(5973, '국군수도병원', '031-725-6075', '031-725-6119')


KeyboardInterrupt: Interrupted by user